In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# ラベル読み込み
p = Path("./facebook/")
featnames_df = pd.DataFrame(columns=["file", "dim", "feat_name", "feat_id"])
for f in p.glob("*.featnames"):
    header = ["dim", "feat_name", "feat_id"]
    df = pd.read_csv(f, header=None, names=header, sep=" ", usecols=[0, 1, 3])
    df["file"] = int(f.stem)
    df = df[["file", "dim", "feat_name", "feat_id"]]
    featnames_df = featnames_df.append(df, ignore_index=True)

featnames_df = featnames_df.sort_values(["file", "dim"]).reset_index(drop=True)


# 今回はfeat_id=77の"gender;anonymized"に着目
df = featnames_df[(featnames_df["feat_id"] == 77)]

gender_feat = pd.DataFrame(columns=["id", "gender"])

for index, row in df.iterrows():
    feat_df = pd.read_csv(p / Path("{}.feat".format(row["file"])), sep=" ", header=None)
    feat_df.columns = ["id"] + list(feat_df.columns)[:-1]
    # id列 と 着目しているfeat_idに対応する列 のみを取り出す
    feat_df = feat_df.loc[:, ["id", row["dim"]]]
    feat_df.columns = gender_feat.columns
    gender_feat = gender_feat.append(feat_df, ignore_index=True)

gender_feat = gender_feat.drop_duplicates().sort_values("id").reset_index(drop=True)
gender_feat = gender_feat.astype({"id": int, "gender": int})

In [3]:
# node2vecで生成したベクトル読み込み
vec_df = pd.read_csv(Path("n2v_out.emd"), sep=" ", header=None)
vec_df.columns = ["id"] + list(vec_df.columns)[:-1]
vec_df = vec_df.sort_values("id").reset_index(drop=True)

In [4]:
# ラベルとベクトルを結合
X_y_df = pd.merge(gender_feat, vec_df, how="inner", on="id")

X_y_df.to_csv("n2v_X_y.txt", sep=" ", index=False)